In [1]:
import pandas as pd
import os
import re
import unicodedata


!pip install rapidfuzz

from rapidfuzz import fuzz, process

In [2]:
base_dir = os.getcwd() 

grados=pd.read_csv("../Input/tabla_grados.csv")

grados['nombre_grado'] = grados['nombre_grado'].str.split('/')
grados = grados.explode('nombre_grado')
grados['nombre_grado'] = grados['nombre_grado'].str.strip()

centros=pd.read_csv("../Input/centros_universitarios.csv")

centros = centros[centros["tipo"].notna()]


def limpiar_universidad(valor):
    if pd.isna(valor):
        return valor
    texto = str(valor)

    if re.search(r'\S+@\S+', texto):
        texto = re.sub(r'^.*?\S+@\S+\s*', '', texto)
    
    texto = re.sub(r'(\(|\)|C/).*$', '', texto)

    return texto.strip()

centros["universidad"] = centros["universidad"].apply(limpiar_universidad)
centros["universidad"] = centros["universidad"].replace(r'\d+', '', regex=True)
centros["universidad"] = centros["universidad"].replace(r'["\']', '', regex=True)


In [3]:
centros.head(2)

,nombre_grado,universidad,tipo,facultad,direccion,telefonos,webs,emails
0,Antropología Social y Cultural,Universidad Nacional de Educación a Distancia ...,Pública,Facultad de Filosofía,"Paseo de la Senda del Rey, 7 (Edificio Humanid...",91 398 69 03,www.uned.es,negociado.alumnos.filosofia@adm.uned.es
1,Bellas Artes,Universidad Antonio de Nebrija,Privada,Facultad de Comunicación y Artes Campus de Mad...,"C/ Santa Cruz de Marcenado, 27 28015 Madrid",91 452 11 03,www.nebrija.com,informa@nebrija.com


## Preprocesar centros

### Agrupar universidades

In [4]:
centros_procesados=centros.copy()

def agrupar_similares(valores, umbral=85, umbral_siglas=85):
    valores_unicos = list(valores.dropna().unique())
    grupos = {}
    usados = set()

    def siglas(texto):
        return ''.join([c for c in texto if c.isupper()])

    for v in valores_unicos:
        if v in usados:
            continue
        grupo = [v]
        usados.add(v)
        siglas_v = siglas(v)

        for otro in valores_unicos:
            if otro in usados:
                continue
            simil = fuzz.token_sort_ratio(v, otro)
            siglas_otro = siglas(otro)

            if simil >= umbral:
                grupo.append(otro)
                usados.add(otro)
                continue

            if siglas_v and siglas_otro:
                simil_siglas = fuzz.ratio(siglas_v, siglas_otro)
                if simil_siglas >= umbral_siglas:
                    grupo.append(otro)
                    usados.add(otro)
                    continue

        cambio = grupo[0]
        for previo in grupo:
            grupos[previo] = cambio

    return grupos


mapa_universidades = agrupar_similares(centros_procesados['universidad'])

universidades_procesadas = pd.DataFrame(list(mapa_universidades.items()), columns=['previo', 'cambio'])

centros_procesados['universidad'] = centros_procesados['universidad'].map(mapa_universidades).fillna(centros_procesados['universidad'])

print("=== UNIVERSIDADES_PROCESADAS ===")
print(universidades_procesadas.to_string(index=False))

=== UNIVERSIDADES_PROCESADAS ===
                                                      previo                                             cambio
          Universidad Nacional de Educación a Distancia UNED Universidad Nacional de Educación a Distancia UNED
Universidad Nacional de Educación a Distancia de Madrid UNED Universidad Nacional de Educación a Distancia UNED
                              Universidad Antonio de Nebrija                     Universidad Antonio de Nebrija
                           Universidad Complutense de Madrid                  Universidad Complutense de Madrid
                            Universidad Francisco de Vitoria                   Universidad Francisco de Vitoria
                                 Universidad Rey Juan Carlos                        Universidad Rey Juan Carlos
                              Universidad Alfonso X El Sabio                     Universidad Alfonso X El Sabio
                                    Universidad San PabloCEU           

### Arreglar facultades

In [5]:
palabras_clave = ['Ctra', 'Carretera', 'Plaza', 'Edificio', 'Ronda']

def limpiar_facultad(texto):
    if not isinstance(texto, str):
        return texto
    patron = r'\b(?:' + '|'.join(palabras_clave) + r'|\d+)\b.*'
    return re.sub(patron, '', texto, flags=re.IGNORECASE).strip()

centros_procesados['facultad'] = centros_procesados['facultad'].apply(limpiar_facultad)

In [6]:
def quitar_tildes(texto):
    if not isinstance(texto, str):
        return texto
    return ''.join(
        c for c in unicodedata.normalize('NFKD', texto) 
        if not unicodedata.combining(c)
    )

centros_procesados['facultad'] = centros_procesados['facultad'].apply(quitar_tildes)

### Arreglar grados

In [7]:
centros_procesados['nombre_grado'] = centros_procesados['nombre_grado'].apply(quitar_tildes)

In [8]:
centros_procesados.head()

,nombre_grado,universidad,tipo,facultad,direccion,telefonos,webs,emails
0,Antropologia Social y Cultural,Universidad Nacional de Educación a Distancia ...,Pública,Facultad de Filosofia,"Paseo de la Senda del Rey, 7 (Edificio Humanid...",91 398 69 03,www.uned.es,negociado.alumnos.filosofia@adm.uned.es
1,Bellas Artes,Universidad Antonio de Nebrija,Privada,Facultad de Comunicacion y Artes Campus de Mad...,"C/ Santa Cruz de Marcenado, 27 28015 Madrid",91 452 11 03,www.nebrija.com,informa@nebrija.com
2,Bellas Artes,Universidad Complutense de Madrid,Pública,Facultad de Bellas Artes Campus de Moncloa,"C/ Pintor el Greco, 2 28040 Madrid",91 394 35 60,www.ucm.es,safba@ucm.es
3,Bellas Artes,Universidad Francisco de Vitoria,Privada,Facultad de Ciencias de la Comunicacion,NaN,91 351 03 03 / 91 709 14 00,www.ufv.es,info@ufv.es
4,Bellas Artes,Universidad Rey Juan Carlos,Pública,Facultad de Artes y Humanidades Campus de Aran...,"C/ San Pascual, s/n 28300 Aranjuez (Madrid)",91 488 44 21,www.urjc.es,aranjuez.secretariadealumnos@urjc.es


## Preprocesar grados

In [9]:
grados_procesados=grados.copy()

### Arreglar grados

In [10]:
grados_procesados['nombre_grado'] = grados_procesados['nombre_grado'].apply(quitar_tildes)

In [11]:
grados_procesados.head()

,rama,area,nombre_grado,descripcion,salidas,centros
0,Artes y Humanidades,Antropología Social y Cultural,Antropologia Social y Cultural,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...,Universidad Nacional de Educación a Distancia ...
1,Artes y Humanidades,Artes,Bellas Artes,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...,Centro de Educación Superior TAI Escuela Unive...
2,Artes y Humanidades,Artes,Conservacion y Restauracion del Patrimonio Cul...,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...,Universidad Complutense de Madrid (Pública) Fa...
3,Artes y Humanidades,Artes,Artes Escenicas,Este grado está adaptado a la realidad profesi...,Estos profesionales tendrán una amplia proyecc...,Universidad Antonio de Nebrija (Privada) Facul...
4,Artes y Humanidades,Artes,Artes Escenicas e Interpretacion Audiovisual,Este grado tiene como objetivo garantizar una ...,"Actor de teatro, de cine, de TV, modelo public...",Centro de Educación Superior TAI Escuela Unive...


# ramas.csv

In [12]:
ramas = pd.DataFrame({
    "id": range(1, len(grados_procesados["rama"].unique()) + 1),
    "nombre": grados_procesados["rama"].dropna().unique()
})

ramas.to_csv("../Output/ramas.csv", index=False, sep=";")

In [13]:
ramas

,id,nombre
0,1,Artes y Humanidades
1,2,Ciencias Sociales y Jurídicas
2,3,Ciencias
3,4,Ciencias de la Salud
4,5,Ingeniería y Arquitectura


# areas.csv

In [14]:
areas = grados_procesados[["area", "rama"]].dropna().drop_duplicates().reset_index(drop=True)
areas.insert(0, "id", range(1, len(areas) + 1))
areas = areas.merge(ramas[["id", "nombre"]], left_on="rama", right_on="nombre", how="left")
areas = areas.rename(columns={"nombre": "nombre_rama"})
areas = areas.rename(columns={"id_x": "id", "area": "nombre", "id_y": "id_rama"})
areas = areas[["id", "nombre", "id_rama"]]

areas.to_csv("../Output/areas.csv", index=False, sep=";")

In [15]:
areas

,id,nombre,id_rama
0,1,Antropología Social y Cultural,1
1,2,Artes,1
2,3,Diseño,1
3,4,Estudios Culturales,1
4,5,Estudios de Asia y África,1
...,...,...,...
70,71,Ingeniería de la Energía,5
71,72,Ingeniería Matemática,5
72,73,Ingeniería de Materiales,5
73,74,Ingeniería en Telecomunicación,5


# universidad.csv

In [16]:
universidades_unicas = centros_procesados["universidad"].dropna().unique()

universidad = pd.DataFrame({
    "id": range(1, len(universidades_unicas) + 1),
    "nombre": universidades_unicas
})

info_universidades = centros_procesados.drop_duplicates(subset=["universidad"])[
    ["universidad", "tipo"]
]

universidad = universidad.merge(
    info_universidades,
    left_on="nombre",
    right_on="universidad",
    how="left"
).drop(columns=["universidad"])

universidad.to_csv("../Output/universidad.csv", index=False, sep=";")

In [17]:
universidad

,id,nombre,tipo
0,1,Universidad Nacional de Educación a Distancia ...,Pública
1,2,Universidad Antonio de Nebrija,Privada
2,3,Universidad Complutense de Madrid,Pública
3,4,Universidad Francisco de Vitoria,Privada
4,5,Universidad Rey Juan Carlos,Pública
5,6,Universidad Alfonso X El Sabio,Privada
6,7,Universidad San PabloCEU,Privada
7,8,Universidad Europea de Madrid,Privada
8,9,Universidad Carlos III de Madrid,Pública
9,10,Universidad Autónoma de Madrid,Pública


# facultad.csv

In [18]:
facultades_unicas = centros_procesados["facultad"].dropna().unique()

facultad = pd.DataFrame({
    "id": range(1, len(facultades_unicas) + 1),
    "nombre": facultades_unicas
})

info_facultades = centros_procesados.drop_duplicates(subset=["facultad"])[
    ["facultad", "universidad", "direccion", "telefonos", "webs", "emails"]
]

facultad = facultad.merge(
    info_facultades,
    left_on="nombre",
    right_on="facultad",
    how="left"
).drop(columns=["facultad"])

facultad=facultad.merge(universidad[["id", "nombre"]], left_on="universidad", right_on="nombre")

facultad=facultad[["id_x", "nombre_x", "direccion", "telefonos", "webs", "emails", "id_y"]]
facultad = facultad.rename(columns={'id_x': 'id', 'nombre_x': 'nombre', 'id_y': 'id_universidad'})
facultad=facultad.drop_duplicates()

facultad=facultad[["id", "nombre", "id_universidad", "direccion", "telefonos", "webs", "emails"]]

facultad.to_csv("../Output/facultad.csv", index=False, sep=";")


In [19]:
facultad

,id,nombre,id_universidad,direccion,telefonos,webs,emails
0,1,Facultad de Filosofia,1,"Paseo de la Senda del Rey, 7 (Edificio Humanid...",91 398 69 03,www.uned.es,negociado.alumnos.filosofia@adm.uned.es
1,21,Facultad de Geografia e Historia,1,"Paseo de la Senda del Rey, 7 28040 Madrid",91 398 67 04,www.uned.es,negociado.geohist@adm.uned.es
2,27,Facultad de Filologia,1,"Paseo de la Senda del Rey, 7 28040 Madrid",91 398 67 03,www.uned.es,negociado.alumnos.filologia@adm.uned.es
3,45,Facultad de Ciencias Politicas y Sociologia,1,"C/ Del Obispo Trejo, 2 28040 Madrid",91 398 89 58,www.uned.es,alumnos.poli@adm.uned.es | infouned@adm.uned.es
4,99,Facultad Ciencias Economicas y Empresariales,1,"Paseo Senda del Rey, 11 28040 Madrid",91 398 63 04,www.uned.es,negociado.economicas@adm.uned.es
...,...,...,...,...,...,...,...
183,77,Facultad de Educacion Campus Villafranca,15,"C/ Castillo de Alarcón, 49 28692 Villanueva de...",91 815 31 31,www.ucjc.edu,info@ucjc.edu
184,82,Facultad de Tecnologia y Ciencia Campus de Alm...,15,"C/ Almagro, 5 28010 Madrid",91 298 01 23,www.ucjc.edu,info@ucjc.edu
185,83,Facultad de Tecnologia y Ciencia,15,"C/ Castillo de Alarcón, 49 Urb. Villafranca de...",91 815 31 31,www.ucjc.edu,info@ucjc.edu
186,86,Facultad de Tecnologia y Ciencia Campus Almagro,15,"C/ Almagro, 5 28010 Madrid",91 298 01 23,www.ucjc.edu,info@ucjc.edu


# grados.csv

In [20]:
grados_unicos = grados_procesados["nombre_grado"].dropna()

grados_tabla = pd.DataFrame({
    "id": range(1, len(grados_unicos) + 1),
    "nombre": grados_unicos
})

info_grados = grados_procesados.drop_duplicates(subset=["nombre_grado"])[
    ["nombre_grado", "area", "descripcion", "salidas"]
]

grados_tabla = grados_tabla.merge(
    info_grados,
    left_on="nombre",
    right_on="nombre_grado",
    how="left"
).drop(columns=["nombre_grado"])

grados_tabla = grados_tabla.merge(
    areas[["id", "nombre"]],
    left_on="area",
    right_on="nombre"
)

grados_tabla = grados_tabla[["id_x", "nombre_x", "id_y", "descripcion", "salidas"]]
grados_tabla = grados_tabla.rename(columns={
    'id_x': 'id',
    'nombre_x': 'nombre',
    'id_y': 'id_area'
})

grados_tabla = grados_tabla.drop_duplicates(subset=["nombre", "id_area"])

grados_tabla["id"] = range(1, len(grados_tabla) + 1)

grados_tabla.to_csv("../Output/grados.csv", index=False, sep=";")

In [21]:
grados_tabla

,id,nombre,id_area,descripcion,salidas
0,1,Antropologia Social y Cultural,1,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
1,2,Antropologia Social y Cultural,15,Esta titulación permite tener un conocimiento ...,Los graduados en esta titulación podrán des em...
4,3,Bellas Artes,2,Los estudios de Bellas Artes tienen entre sus ...,Las profesiones que estos titulados pueden eje...
5,4,Conservacion y Restauracion del Patrimonio Cul...,2,El Grado en Conservación y Restauración del Pa...,El ámbito de trabajo profesional del Conservad...
6,5,Artes Escenicas,2,Este grado está adaptado a la realidad profesi...,Estos profesionales tendrán una amplia proyecc...
...,...,...,...,...,...
224,217,Ciencia de Datos e Inteligencia Artificial,75,El grado cubre la creciente necesidad de perfi...,Dichos profesionales estarán capacitados para ...
225,218,Inteligencia Artificial,75,Este Grado tiene como finalidad proporcionar a...,Estratega de entorno digital Especialista en c...
226,219,Computacion e Inteligencia Artificial,75,Este Grado tiene como finalidad proporcionar a...,Estratega de entorno digital Especialista en c...
227,220,Ingenieria Robotica Sotware,75,El objetivo de estas titulaciones es formar pr...,La robótica se ha utilizado desde hace años en...


# grados_ofertados.csv

In [22]:
grados_ofertados=centros_procesados[["nombre_grado", "facultad"]]

grados_ofertados=grados_ofertados.merge(grados_tabla[["id", "nombre"]], left_on="nombre_grado", right_on="nombre")
grados_ofertados = grados_ofertados.rename(columns={'id': 'id_grado'})

grados_ofertados=grados_ofertados[['nombre_grado',"facultad", "id_grado"]]

grados_ofertados=grados_ofertados.merge(facultad[["id", "nombre"]], left_on="facultad", right_on="nombre")
grados_ofertados = grados_ofertados.rename(columns={'id': 'id_facultad'})

grados_ofertados[grados_ofertados['nombre_grado'].str.lower() == "matemática computacional"]

grados_ofertados=grados_ofertados[["id_grado", "id_facultad"]]
grados_ofertados=grados_ofertados.drop_duplicates()

grados_ofertados.insert(0, "id", range(1, len(grados_ofertados) + 1))

grados_ofertados.to_csv("../Output/grados_ofertados.csv", index=False, sep=";")

In [23]:
grados_ofertados

,id,id_grado,id_facultad
0,1,1,1
1,2,2,1
2,3,30,1
3,4,1,15
4,5,2,15
...,...,...,...
544,488,211,184
545,489,213,186
546,490,214,186
547,491,215,187
